## STT Sample_01 (오디오 텍스트 변환 요청하기)

In [1]:
# Imports the Google Cloud client library
from google.cloud import speech

# Instantiates a client
client = speech.SpeechClient()

# The name of the audio file to transcribe
gcs_uri = "gs://cloud-samples-data/speech/brooklyn_bridge.raw"

audio = speech.RecognitionAudio(uri=gcs_uri)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code="en-US",
)

# Detects speech in the audio file
response = client.recognize(config=config, audio=audio)

for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

Transcript: how old is the Brooklyn Bridge


## STT Sample_02 (짧은 오디오 파일을 텍스트로 변환)

In [2]:
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    import io

    client = speech.SpeechClient()

    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
        sample_rate_hertz=48000,
        language_code="en-US",
    )

    response = client.recognize(config=config, audio=audio)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u"Transcript: {}".format(result.alternatives[0].transcript))


In [3]:
transcribe_file('data/slang_eng.mp3')

Transcript: what if we go through the different methods of using the word fuck
Transcript:  so
Transcript:  fuck you alright fuck off
Transcript:  yeah and I can be positive I can love you
Transcript:  yeah I fuck with you I want to punch you


# Slang Filter 구현

In [4]:
pip install pydub

You should consider upgrading via the 'c:\users\dorot\pycharmprojects\kagglestudy\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
pip install io

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
You should consider upgrading via the 'c:\users\dorot\pycharmprojects\kagglestudy\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
from pydub import AudioSegment
import numpy as np

## STT

In [7]:
def transcribe_file(speech_file):
    """Transcribe the given audio file."""
    from google.cloud import speech
    import io

    client = speech.SpeechClient()

    with io.open(speech_file, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED, # Encoding 변경함(LINEAR16->ENCODING_UNSPECIFIED)
        sample_rate_hertz=48000,       # 오디오 파일의 sample rate에 맞도록 조정(48.000kHz)
        language_code="en-US",         # 한국어일 경우, "ko-KR"로 변경
        enable_word_time_offsets=True, # 단어별로 그 단어가 나온 시간을 찍어줌
        use_enhanced=True              # 더 향상된 STT 모델을 사용할 수 있도록
    )

    response = client.recognize(config=config, audio=audio)
    
    '''
    timeline: 단어가 시작하는 시간과 끝나는 시간 (단위: millisecond)
    swear_timeline: 욕설이 시작하는 시간과 끝나는 시간 (단위: millisecond)
    words: STT를 통해 텍스트로 변환된 모든 단어 저장
    '''
    timeline, swear_timeline, words = [], [], []

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    for result in response.results:
        # The first alternative is the most likely one for this portion.
        print(u"Transcript: {}".format(result.alternatives[0].transcript))
            
        for word in result.alternatives[0].words:
            # 단어가 시작하는 시간과 끝나는 시간을 ms 단위로 저장
            timeline.append([
                int(word.start_time.seconds * 1000 + word.start_time.microseconds * (10**-6)),
                int(word.end_time.seconds * 1000 + word.end_time.microseconds * (10**-6))
            ])
            
            # 텍스트로 변환된 모든 단어 저장
            words.append(word.word)
            
            # 단어에 'fuck'이라는 단어가 포함되어있다면, swear_timeline에 그 단어의 timeline을 따로 저장
            if 'fuck' in word.word:
                swear_timeline.append([
                    int(word.start_time.seconds * 1000 + word.start_time.microseconds * (10**-6)),
                    int(word.end_time.seconds * 1000 + word.end_time.microseconds * (10**-6))
                ])
                
    return timeline, swear_timeline, words

# Execute
timeline, swear_timeline, words = transcribe_file('data/slang_eng.mp3')

print(timeline)
print(words)
print(swear_timeline)


Transcript: what if we go through the different methods of using the word fuck
Transcript:  so
Transcript:  fuck you alright fuck off
Transcript:  yeah and I can be positive I can love you
Transcript:  yeah I fuck with you I want to punch you
[[2000, 3000], [3000, 3000], [3000, 3000], [3000, 3000], [3000, 4000], [4000, 4000], [4000, 4000], [4000, 5000], [5000, 5000], [5000, 5000], [5000, 6000], [6000, 6000], [6000, 6000], [7000, 8000], [9000, 9000], [9000, 12000], [12000, 13000], [13000, 14000], [14000, 14000], [19000, 20000], [20000, 20000], [20000, 20000], [20000, 21000], [21000, 21000], [21000, 21000], [21000, 21000], [21000, 21000], [21000, 22000], [22000, 22000], [24000, 24000], [24000, 25000], [25000, 25000], [25000, 25000], [25000, 25000], [25000, 26000], [26000, 26000], [26000, 26000], [26000, 27000], [27000, 27000]]
['what', 'if', 'we', 'go', 'through', 'the', 'different', 'methods', 'of', 'using', 'the', 'word', 'fuck', 'so', 'fuck', 'you', 'alright', 'fuck', 'off', 'yeah', '

In [8]:
len(words)

39

In [9]:
len(timeline)

39

In [10]:
len(swear_timeline)

4

STT를 통해 변환된 단어의 총 갯수는 39개이며, 이 중 'fuck'이 포함된 단어는 4개인 것을 확인할 수 있다.

## Load Original Audio File

In [11]:
pip install ffprobe

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\dorot\pycharmprojects\kagglestudy\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
print(AudioSegment.ffmpeg)

ffmpeg


#### AudioSegment.from_file로 파일 불러오려면 FileNotFoundError 발생
-> [해결방법](https://stackoverflow.com/questions/68247327/cannot-find-a-solution-to-winerror-2-file-not-found-by-audiosegment-from-mp3)
1. ffmpeg.exe, ffprobe.exe,  ffplay.exe 설치 후 동일위치에 저장
2. AudioSegment.from_file('파일명', format='mp3') 대신에 AudioSegment.from_mp3('파일명') 으로 작성

In [13]:
AudioSegment.converter = r'C:\Users\dorot\PycharmProjects\KaggleStudy\TP_07_Audio Profanity Filter\ffmpeg.exe'                        
AudioSegment.ffprobe   = r'C:\Users\dorot\PycharmProjects\KaggleStudy\TP_07_Audio Profanity Filter\ffprobe.exe'

In [14]:
# pydub을 이용하여 오디오 원본파일을 sound에 저장
# sound = AudioSegment.from_file('data/slang_eng.mp3', format='mp3')
sound = AudioSegment.from_mp3('data/slang_eng.mp3')

print(len(sound))

30955


'slang.mp3'파일의 길이는 30.955초

## Create Beep Sound

In [15]:
def create_beep(duration):
    sps = 44100
    freq_hz = 1000.0
    vol = 0.5
    
    esm = np.arange(duration / 1000 * sps)
    wf = np.sin(2 * np.pi * esm * freq_hz / sps)
    wf_quiet = wf * vol
    wf_int = np.int16(wf_quiet * 32767)
    
    beep = AudioSegment(wf_int.tobytes(),
                        frame_rate = sps,
                        sample_width = wf_int.dtype.itemsize,
                        channels = 1 
                       )
    return beep

In [16]:
# beep 사운드를 1초동안 생성시키기
beep = create_beep(duration = 1000)
beep

## Overlay Partially

In [17]:
# 'fuck'이 나오는 첫번째 단어에 beep sound를 입히기

'''
AudioSegment.overlay() : 오디오A 위에 다른 오디오B를 덧입힐 수 있다.

A.overlay(B, position, gain_during_overlay)

A: 오리지널 오디오
B: A위에 입힐 오디오
position: 덧입혀질 위치
gain_during_overlay: 오리지널 오디오의 볼륨을 얼만큼 줄일것인지
'''

i = 0   # 첫번째 단어에 !
mixed = sound.overlay(beep, position = swear_timeline[i][0], gain_during_overlay = -20)
mixed

## Result

In [18]:
mixed_final = sound  # 오리지널 오디오

for i in range(len(swear_timeline)):
    '''
    beep sound 길이는 swear_timeline[i][1] - swear_timeline[i][0] 으로 줄 수 있으나,
    'fuck' 의 sound 구간을 보면 [9000, 9000]으로 간격이 없으므로 임의로 duration값을 고정해준다
    '''
    beep = create_beep(duration = 500)  # beep sound 길이
    mixed_final = mixed_final.overlay(beep, position = swear_timeline[i][0], gain_during_overlay = -20)
    
mixed_final    

한국어 사운드로 테스트했을때, STT 성능이 좋지 않아서 영어 사운드로 테스트를 해 보았으나,  
마찬가지로 많은 단어가 text로 변환되지 않은 것을 확인할 수 있었다.<br><br>

찾아보니, STT는 오디오에 노이즈가 있을 경우 정확도가 떨어지기 때문에  
품질이 좋은 마이크를 사용하거나, 마이크를 화자와 최대한 가깝게 배치하여 녹음된 선명한 오디오를 사용하는 것을 권장한다고 한다.<br><br>

기대했던 것 보다는 인식률이 좋지 않아서 아쉬웠지만,  
그래도 google의 STT를 경험해볼 수 있어서 뜻깊었고, 음원 데이터는 처음 접해봤는데 생각보다 재미있었다.

# Export

In [19]:
# AudioSegment를 mp3 파일로 내보내기
mixed_final.export('result/result_fxck.mp3', format='mp3')

<_io.BufferedRandom name='result/result_fxck.mp3'>